In [1]:
import pandas as pd
import numpy as np
import csv
import pickle
from collections import Counter
import matplotlib.pyplot as plt
import powerlaw,math
import os
import scipy.stats as stats
import random
from numpy import polyfit, poly1d

In [214]:
files = os.listdir('chemistry pubs')

In [215]:
TP = {}
for k in files:
    try:
        with open('chemistry pubs/'+k,'rb')as f:
            tp = pickle.load(f)
        for i in tp:
            TP[i] = tp[i]
    except:
        print(k)

In [216]:
pub_chemistry0 = TP
print(len(pub_chemistry0))

49248


In [217]:
#将生涯长度为0的做调整
pub_chemistry = {}
empty=[]
nopa = []
for k in pub_chemistry0:
    tp = pub_chemistry0[k]
    if len(tp)>0:
        cl = tp[2]
        if cl==1:
            empty.append(k)
            tp[3] = tp[0]  # 0总发文量
            tp[6] = tp[5]  # 5总引用
            tp[9] = tp[8]  # 8总合作者数
        else:
            tp = tp
    else:
        nopa.append(k)
        tp = tp
    pub_chemistry[k] = tp

In [280]:
# neuro = pd.read_csv('chemistry_1224.csv')
neuro = pd.read_csv('chemistry_0123.csv')   #此处已经是下面代码更新后的数据

In [281]:
neuro.shape

(25138, 13)

In [282]:
Neuro = pd.read_csv('chemistry.csv')
MTO = np.array(Neuro.MentorID)

In [288]:
StopYear = neuro.StopYear
print(min(StopYear))
print(max(StopYear))

1901
1995


In [293]:
# 在进行退出率的相关计算时，以所有的学生数为参考
TP = []
for k in neuro.MenteeID:
    if k in MTO:
        TP.append(1)
    else:
        TP.append(0)
neuro['Survival0'] = TP              
# neuro = neuro[neuro.StopYear<2000]

In [294]:
#测试大小团队中的生存率
all_big = neuro['MenteeID'][neuro.BigOrSmall==1]
all_small = neuro['MenteeID'][neuro.BigOrSmall==0]
sb = neuro['MenteeID'][neuro.Survival0==1]
sur_big = all_big[all_big.isin(sb)]
sur_small = all_small[all_small.isin(sb)]
print(len(sur_big)/len(all_big))
print(len(sur_small)/len(all_small))

0.5683373110580748
0.7861120581884306


In [297]:
#在进行学生表现的相关计算时，以前20年是否招到学生作为活下的参照
TP = []
for k in neuro.NumOfMenteesOfTheMenteeIn20:
    if k>0:
        TP.append(1)
    else:
        TP.append(0)
neuro['Survival1'] = TP              

In [301]:
#补充老师的文章属性信息
attri = ['TotalPubsOfMentor','YearOfTheFirstPubsOfMentorPubs','YearlyPubsOfMentor','TotalCitationOfMentor',
         'YearlyCitationOfMentor','TotalCollaOfMentor','YearllyCollaOfMentor',
        ]
index = [0,1,3,5,6,8,9]
for i in range(len(attri)):
    atr = attri[i]
    if atr!='YearOfTheFirstPubsOfMentorPubs':
        TP = []
        for k in neuro.MentorID:
            try:
                tp = pub_chemistry[k][index[i]]
            except:
                tp = 0                            #在 mag 中没有找到该作者的发表信息
            TP.append(tp)
    else:
        TP = []
        for k in neuro.MentorID:
            try:
                tp = pub_chemistry[k][index[i]]
            except:
                tp = -1                            #在 mag 中没有找到该作者的发表信息,则第一篇文章发表年份记为 -1  
            TP.append(tp)
    neuro[atr] = TP 

In [302]:
#补充学生的文章属性信息
attri = ['YearOfTheFirstPubsOfMenteePubs','First5YearPubsOfMentee','First5YearCitationOfMentee',
         'First5YearCollaOfMentee','YearlyPubsOfMentee','YearlyCitationOfMentee']
index = [1,4,7,10,3,6]
for i in range(len(attri)):
    atr = attri[i]
    if atr!='YearOfTheFirstPubsOfMenteePubs':
        TP = []
        for k in neuro.MenteeID:
            try:
                tp = pub_chemistry[k][index[i]]
            except:
                tp = 0                            #在 mag 中没有找到该作者的发表信息
            TP.append(tp)
        
    else:
        TP = []
        for k in neuro.MenteeID:
            try:
                tp = pub_chemistry[k][index[i]]
            except:
                tp = -1                            #在 mag 中没有找到该作者的发表信息
            TP.append(tp)
        
    neuro[atr] = TP 

In [303]:
#通过学生的第一篇文章发表时间补充学生的出道的时间
TP = []
tp0 = list(neuro.StopYear)                        # StopYear 中又可能出现 -1
tp1 = list(neuro.YearOfTheFirstPubsOfMenteePubs)  #这里面也可能出现 -1
for i in range(len(tp0)):
    if tp0[i]>0 and tp1[i]>0:
        TP.append(min(tp0[i],tp1[i]))
    elif tp0[i]*tp1[i]<0:
        TP.append(max(tp0[i],tp1[i]))
    else:
        TP.append(-1)
neuro['BirthYearOfMentee'] = TP  

In [304]:
#通过老师第一篇文章发表时间补充每位学生受训时的老师的学术年龄
TP = []
tp0 = list(neuro.StopYear)
tp1 = list(neuro.YearOfTheFirstPubsOfMentorPubs)
for i in range(len(tp1)):
    if tp0[i]>0 and tp1[i]>0:
        TP.append(tp0[i]-float(tp1[i]))
    else:
        TP.append(-1)
neuro['CareerAgeOfMentorInTraining'] = TP # -1代表无法判断老师的学术年龄 

In [305]:
neuro.shape

(25138, 30)

In [306]:
neuro.head(1)

,Unnamed: 0,CID,MenteeID,MentorID,MentorshipType,Institution,InstitutionMAGID,StartYear,StopYear,NumOfMenteesOfTheMentor,...,TotalCollaOfMentor,YearllyCollaOfMentor,YearOfTheFirstPubsOfMenteePubs,First5YearPubsOfMentee,First5YearCitationOfMentee,First5YearCollaOfMentee,YearlyPubsOfMentee,YearlyCitationOfMentee,BirthYearOfMentee,CareerAgeOfMentorInTraining
0,3,6,18761,9,1,"University of California, Berkeley",95457486.0,-1,1984,10,...,139,3.564103,1986.0,19,745,25,7.666667,441.083333,1984.0,21.0


##### 加入受训期间老师的发文量和学生与导师的合作文章数量

In [307]:
neuro = neuro.drop(['Unnamed: 0'],axis=1)

In [308]:
chemistry_colla_info = pd.read_csv('chemistry_colla_info.csv')    #在 collaboration code 中计算出来的 

In [309]:
CC = {}
for k in np.array(chemistry_colla_info):
    tp_key = (k[2],k[0])
    tp_value = (k[1],k[3])
    CC[tp_key] = tp_value

In [311]:
TP0 = []
TP1 = []
NN = np.array(neuro)
for i in range(len(neuro)):
    try:
        tp_key = (NN[i][1],NN[i][2])
#         print(tp_key)
        temp = CC[tp_key]
        
        TP0.append(temp[0])
        TP1.append(temp[1])
    except:
        TP0.append(0)
        TP1.append(0)
neuro['PubsOfMentorInTraning'] = TP0 
neuro['CollaPubsOfWithMentor'] = TP1

In [312]:
neuro.shape

(25138, 31)

### 导出最后计算的数据

In [313]:
# neuro.to_csv('chemistry_0123.csv',header=neuro.columns,index=0)

In [315]:
neuro.head(3)

,CID,MenteeID,MentorID,MentorshipType,Institution,InstitutionMAGID,StartYear,StopYear,NumOfMenteesOfTheMentor,NumOfMenteesOfTheMentor10,...,YearOfTheFirstPubsOfMenteePubs,First5YearPubsOfMentee,First5YearCitationOfMentee,First5YearCollaOfMentee,YearlyPubsOfMentee,YearlyCitationOfMentee,BirthYearOfMentee,CareerAgeOfMentorInTraining,PubsOfMentorInTraning,CollaPubsOfWithMentor
0,6,18761,9,1,"University of California, Berkeley",95457486.0,-1,1984,10,4,...,1986.0,19,745,25,7.666667,441.083333,1984.0,21.0,3,0
1,161,12561,52639,1,"University of Washington, Seattle",201448701.0,1967,1971,1,1,...,1972.0,11,305,25,3.435897,98.384615,1971.0,15.0,5,1
2,212,188,189,1,Harvard University,136199984.0,-1,1927,9,4,...,1928.0,5,702,2,0.391304,32.717391,1927.0,26.0,8,0
